## Librerías

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

## Carga de datos

In [3]:
#### Conectar #######
conn=sql.connect('data/db_movies')
cur=conn.cursor()

In [4]:
#ver tablas
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movie_final',),
 ('full_ratings',)]

## 3 Sistema de recomendación basado en contenido KNN Con base en todo lo visto por el usuario

### Top 10 de las películas más similares a las vistas por el usuario y con una que le haya dado una calificación mayor a 4.

### Top 5 de las películas con el género más visto por el usuario

## 4 Sistema de recomendación filtro colaborativo